## TASKS
```
DESCRIPTION                             STATUS 
- Load seed data into DB.               done
- Clean and transform scraped data      done
- Create an idempotent etl job          ongoing
```

In [1]:
import pandas as pd
from pathlib import Path
import os
from datetime import datetime
import logging

In [2]:
# Setup logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


In [3]:
scraped_data_source = r"C:\Users\APIN PC\OneDrive\Documents\DS\DE_Inter\data_epic_capstone\etl\data\29-05-2025_ai_tools_scraped.json"
seed_data_source = r"C:\Users\APIN PC\OneDrive\Documents\DS\DE_Inter\data_epic_capstone\etl\data\seeded_ai_agents.csv"

## Data Loading

In [4]:
def read_data(source_path: str) -> pd.DataFrame:
    """

    Args:
        source_path (str): Data Path

    Raises:
        ValueError: Raises error for unsupported data type.

    Returns:
        dataframe: Pandas Dataframe.
    """

    try:
        ext = Path(source_path).suffix
        if ext == ".csv":
            return pd.read_csv(source_path)
        elif ext == ".json":
            return pd.read_json(source_path)
        elif ext == ".parquet":
            return pd.read_parquet(source_path)
        logger.info("Data successfully read!")
    except Exception as e:
        logger.error(f"Error: {e}. Unsupported file format! Use csv, json or parquet.")


## Data Preview

In [5]:
scraped_df = read_data(scraped_data_source)
seed_df = read_data(seed_data_source)

scraped_df.info()
seed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         378 non-null    object
 1   description  378 non-null    object
 2   url          376 non-null    object
 3   tags         377 non-null    object
 4   pricing      377 non-null    object
 5   page         378 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 17.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          72 non-null     object
 1   description   72 non-null     object
 2   homepage_url  72 non-null     object
 3   category      72 non-null     object
 4   source        72 non-null     object
 5   created_at    72 non-null     object
 6   updated_at    72 non-null     object
 7   trending      72 non-null     ob

## Cleaning

Cleaning "tags" column
- separate the values in the list and choose the unique tag. 
- Each tag must be just a value. (i.e list of len 1)
    + no # in value 
    + no duplicate.

In [6]:
def baseline_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    try:
        df = df.drop(columns=[col for col in ['pricing', 'page'] if col in df.columns])
        new_df = df.dropna()
        new_df = new_df.reset_index(drop=True)
        logger.info("Columns dropped and null values dropped.",
                    extra={
                     "Cols dropped": ['pricing', 'page'],
                     "Null Values Dropped": len(df) - len(new_df)
                    }
                    )
    except Exception as e:
        logger.error(f"Error Raised {e}! Is the input a dataframe? Use a pandas dataframe.",  exc_info=True)
    return df


def remove_hashtags(tags):
    try:
        if isinstance(tags, list):
            clean = [tag for tag in tags if '#' not in tag]
        elif isinstance(tags, str):
            clean = [tags] if "#" not in tags else []
        else:
            clean = []
        clean = ','.join(clean)

        if len(clean) < 4:
            clean = clean.upper()
        else:
            clean = clean.lower().capitalize()
    except Exception as e:
        logger.error(f"Error Raised at tags column cleaning {e}! Use tags column.",  exc_info=True)
    return clean


def clean_data(df):
    try:
        # df = baseline_cleaning(df=scraped_df)
        df = df.drop(columns=[col for col in ['pricing', 'page'] if col in df.columns])
        new_df = df.dropna()
        new_df = new_df.reset_index(drop=True)

        if 'tags' in df.columns:
            new_df['tags'] = new_df['tags'].apply(remove_hashtags)
        else:
            pass

        logger.info("Columns dropped and null values dropped.",
                    extra={
                     "Cols dropped": ['pricing', 'page'],
                     "Null Values Dropped": len(df) - len(new_df)
                    }
                    )
        logger.info("Tags Column Successfully cleaned.")
        logger.info("Data successfully cleaned!")
    except Exception as e:
        logger.error(f"Error Raised at full cleaning process: {e}!",  exc_info=True)
    return new_df




## Transformation

In [7]:
def get_created_at(filepath: str) -> str:
    try:
        created_timestamp = os.path.getctime(filepath)
        created_date = datetime.fromtimestamp(created_timestamp)
    except Exception as e:
        logger.error(f"Error Raised: {e}!", exc_info=True)
    return created_date.strftime("%Y-%M-%d")


def transform_data(df: pd.DataFrame, source = None) -> pd.DataFrame:
    try:
        created_day = get_created_at(scraped_data_source)
        if 'source' in df.columns:
            if df['source'] is not None:
                pass
            else:
                df['source'] = source
        else:
            df['source'] = source


        if 'created_at' in df.columns:
            if df['created_at'] is not None:
                pass
            else:
                df['created_at'] = created_day
        else:
            df['created_at'] = created_day
        if 'updated_at' in df.columns:
            if df['updated_at'] is not None:
                pass
            else:
                df['updated_at'] = None
        else:
            df['updated_at'] = None
        

        if 'trending' not in df.columns:
            df['trending'] = None
            df['trending'] = df['trending'].notna().astype(bool)
        else:
            df["trending"] = df["trending"].apply(
                lambda x: False if x == 'Low' else True
                )
            df['trending'] = df['trending'].astype(bool)
    
        trans_df = df.rename(columns={'url': 'homepage_url', 'tags': 'category'})
        
        trans_df['created_at'] = pd.to_datetime(trans_df['created_at'], format="%Y-%M-%d", errors="coerce")
        trans_df['updated_at'] = pd.to_datetime(trans_df['updated_at'], format="%Y-%M-%d", errors="coerce")

        logger.info("Data successfully transformed!")
    except Exception as e:
        logger.error(f"Error Raised at transformation: {e}!", exc_info=True)
    return trans_df

In [ ]:
def merging_dfs(new_df, existing_df) -> pd.DataFrame:
    """
    Merging DFs to extract unique ai_tools
    Returns:
        pd.DataFrame: Merged DF with unique Ai tools
    """
    try:
        merged_df = pd.merge(existing_df, new_df, how="outer", suffixes="_existing")
        merged_df.drop_duplicates(subset=[
            "name", "homepage_url"
            ], inplace=True)
        merged_df = merged_df.reset_index(drop=True)
        logger.info("Existing DB Data and Scraped Data successfully merged!")
    except Exception as e:
        logger.error("Error merging DFs: %s", e, exc_info=True)
    return merged_df

## ETL

In [9]:
def run_basic_etl() -> pd.DataFrame:
    # Extract
    scraped_df = read_data(scraped_data_source)

    # Clean
    clean_scraped_df = clean_data(scraped_df)

    # Transform
    trans_scraped_df = transform_data(
        clean_scraped_df, source="https://aitoolsdirectory.com/"
    )

    return trans_scraped_df
    

In [10]:
comp_df = run_basic_etl()

2025-05-30 12:23:45,994 - INFO - Columns dropped and null values dropped.
2025-05-30 12:23:45,996 - INFO - Tags Column Successfully cleaned.
2025-05-30 12:23:46,003 - INFO - Data successfully cleaned!
2025-05-30 12:23:46,024 - INFO - Data successfully transformed!


## Idempotent ETL Job

In [ ]:
"""
Ai_tools ETL Local DB Setup and data upload

Name: Arowosegbe Victor Iyanuoluwa\n
Email: Iyanuvicky@gmail.com\n
GitHub: https://github.com/Iyanuvicky22/projects
"""

import os
from dotenv import load_dotenv
from sqlalchemy import (
    create_engine,
    Column,
    String,
    Text,
    Boolean,
    DateTime,
    func,
    Integer,
)
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd

load_dotenv(dotenv_path=".env")

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

DB_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

Base = declarative_base()


def connect_db():
    """
    Database connector
    """
    try:
        engine = create_engine(DB_URL)
        Session = sessionmaker(bind=engine, autoflush=False)
        Base.metadata.create_all(engine)
        logger.info("Database succesfully connected to.")
    except SQLAlchemyError as e:
        logger.error("Databse connection error: %s", e, exc_info=True)
    return Session, engine


class Agent(Base):
    """
    Agents table model creation
    Args:
        Base (): SQLAlchemy Base model
    """

    __tablename__ = "agents"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False, index=True)
    description = Column(Text)
    homepage_url = Column(String)
    category = Column(String)
    source = Column(String)
    trending = Column(Boolean, default=False)
    created_at = Column(DateTime, server_default=func.now(), nullable=False)
    updated_at = Column(
        DateTime, server_default=func.now(), onupdate=func.now(), nullable=False
    )


def load_data(df: pd.DataFrame):
    """
    Function to load data into the Database (PostgreSQL).
    Args:
        df (pd.DataFrame): Cleaned and Transformed data to be loaded.
    """
    Session, engine = connect_db()
    data = df

    def change_to_none(row):
        row.replace(pd.NaT, '')
        return row


    with Session.begin() as session:
        for _, row in data.iterrows():
            ai_tool = session.query(Agent).filter_by(name=str(row["name"])).first()

            if not ai_tool:
                agent = Agent(
                    name=str(row["name"]),
                    description=str(row["description"]),
                    homepage_url=row["homepage_url"],
                    category=row["category"],
                    source=row["source"],
                    trending=row["trending"],
                    created_at=row["created_at"],
                    updated_at=row["updated_at"],
                )
                session.add(agent)
        session.commit()
        logger.info("Data successfully loaded in database!")


In [12]:
def trans_load_seed_df():
    """
    Function to Load Seed Data into Database
    """
    data = read_data(source_path=seed_data_source)

    trans_seed_df = transform_data(df=data)

    load_data(trans_seed_df)

    return trans_seed_df

In [ ]:
trans_load_seed_df()

In [ ]:
from sqlalchemy import select

def fetch_db_records():
    session, engine = connect_db()

    with engine.connect() as conn:
        db_df = pd.read_sql("SELECT * from agents", con=conn)
        conn.commit()
    return db_df

db_df = fetch_db_records()


2025-05-30 13:27:17,157 - INFO - Database succesfully connected to.


In [23]:
db_df.trending.value_counts()

trending
True     60
False    12
Name: count, dtype: int64

## Functions to Work On

In [ ]:
def delta_check(new_df, existing_df):
    # Merge and check for differences
    merged = new_df.merge(
        existing_df, on=["name", "homepage_url"], how="left", suffixes=("", "_existing")
    )
    changed = merged[
        (merged["name"] != merged["name_existing"])
        | (merged["homepage_url"] != merged["homepage_url_existing"])
    ]
    return changed[new_df.columns] 



def upsert_records(conn, df):
    cursor = conn.cursor()
    for _, row in df.iterrows():
        cursor.execute(
            """
            INSERT INTO agents (name, homepage_url, email, phone)
            VALUES (?, ?, ?, ?)
            ON CONFLICT(name, homepage_url)
            DO UPDATE SET email=excluded.email, phone=excluded.phone
        """,
            (row["name"], row["homepage_url"], row["email"], row["phone"]),
        )
    conn.commit()


def etl_job(source_path):
    df = read_data(source_path)

    if needs_transformation(df):
        df = transform_data(df)

    with engine.connect("agents.db") as conn:
        existing_df = fetch_existing_records(conn)
        delta_df = delta_check(df, existing_df)

        if not delta_df.empty:
            upsert_records(conn, delta_df)
            print(f"Upserted {len(delta_df)} records.")
        else:
            print("No changes detected. Idempotent run.")


## Notes

### Pending tasks
- Load Seed data separately.    ```done```
- Load scraped data and check for duplicates with name(lower) and homepage_url.
- Write tests to check for:
    + test for duplicates.
    + test for invalid rows.
    + test for correct upserts.
